In [1]:
import time
import sys
from subprocess import call
import cv2
import matplotlib.pyplot as plt
from IPython.display import display, Image
import ipywidgets as widgets
import threading
import numpy as np
import keras
from qkeras.utils import _add_supported_quantized_objects

In [2]:
# UPLOAD MODEL
# path to model relative to downsample_tb.py
model_file_path = "./model_best.h5"
co = {}
_add_supported_quantized_objects(co)
model = keras.models.load_model(model_file_path, custom_objects = co)

# # print model summary
# print(model.summary())

In [3]:
# take any input img, do largest square center crop if not input not square,
# and return a square img of dimsxdims
def downsamp(img, dims):
    smallest_dim = np.min(np.array([img.shape[0], img.shape[1]]))

    # center crop relative to this image, so that we have square image
    img_sq = img[int(img.shape[0]/2)-int(smallest_dim/2):int(img.shape[0]/2)+int(smallest_dim/2),
                int(img.shape[1]/2)-int(smallest_dim/2):int(img.shape[1]/2)+int(smallest_dim/2),
                :]

    # interpolate to make image 32x32x3
    res = cv2.resize(img_sq, dsize=(dims, dims), interpolation=cv2.INTER_NEAREST)
    
    return res

In [4]:
icon_mask = np.load("icon_mask.npy")
# plt.imshow(icon_mask[2])

In [5]:
# RUN THE MODEL ITERATIVELY AND EVALUATE THE FRAME USING NEURAL NET
# PRESS STOP TO PREVENT THE MODEL FROM CONTINUING
# MODEL PREDICTION SHOWN AS AN ICON ON THE TOP LEFT OF THE SCREEN

# Stop button
# ================
stopButton = widgets.ToggleButton(
    value=False,
    description='Stop',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='square' # (FontAwesome names without the `fa-` prefix)
)


# Display function
# ================
def view(button):
    cap = cv2.VideoCapture(1)
    display_handle=display(None, display_id=True)
    i = 0
    while True:
        _, frame = cap.read()
        frame = cv2.flip(frame, 1) # if your camera reverses your image
        
        '''
        send the image to the accelerator here
        '''
        res = downsamp(frame, 32)
        
        # convert to input ready for neural net
        X_dtest = np.ascontiguousarray(res, dtype=np.float32)  # doesn't change shape, just turns every element to float instead of int
        X_dtest = X_dtest/256.

        # have model predict the image
        y_pred = model.predict(np.array([X_dtest]))
        
        # print 

        mask = icon_mask[np.argmax(y_pred[0])]
        
        # print("frame size is :", frame.shape)
        # print("mask size is : ", mask.size)
        frame = cv2.subtract(frame, mask)
        _, frame = cv2.imencode('.jpeg', frame)
        display_handle.update(Image(data=frame.tobytes()))
        if stopButton.value==True:
            cap.release()
            display_handle.update(None)

            
# Run
# ================
display(stopButton)
thread = threading.Thread(target=view, args=(stopButton,))
thread.start()

ToggleButton(value=False, button_style='danger', description='Stop', icon='square', tooltip='Description')

None